In [2]:
import tensorflow as tf

In [3]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(-78.4243, shape=(), dtype=float32)


In [4]:
print(tf.config.list_physical_devices('CPU'))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [6]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train/train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 47391 files belonging to 26 classes.
